# AdaBoost

Codigo fuente: https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html#sphx-glr-auto-examples-ensemble-plot-adaboost-regression-py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import make_scorer, roc_auc_score

In [2]:
import pandas as pd
import os

In [3]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [4]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

In [5]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [6]:
rng = np.random.RandomState(1)

In [7]:
%run $set_de_entrenamiento_testing_y_prediccion

In [8]:
training = labels_training.merge(df_features, how='inner', on='person')

In [9]:
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

regr_1.fit(training.drop('label', axis=1), training['label'])
regr_2.fit(training.drop('label', axis=1), training['label'])


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x1a184c9168>)

In [10]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

df_features

In [11]:
labels_test['label_predicted'] = regr_1.predict(labels_test.drop('label', axis=1))

In [12]:
%run $calcular_auc

auc score: 0.8260535310497885


In [13]:
labels_test['label_predicted'] = regr_2.predict(labels_test.drop(['label','label_predicted'], axis=1))

In [14]:
%run $calcular_auc

auc score: 0.8574971996566308


# Tuning an AdaBoost regressor

In [15]:
labels_with_features = labels.merge(df_features, how='inner', on='person')
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

In [16]:
splits = 20 # cantidad de splits en el cross validation
n_iter_search = 5 # cantidad de combinaciones, en total splits*n_iter_search RF a probar

In [17]:
from sklearn.model_selection import RandomizedSearchCV


El mejor AdaBoost fue:

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

# Seguir mejorando

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

param_dist = {
 'n_estimators': list(range(10,400,10)),
 'learning_rate' : np.arange(0.01,0.2,0.04),
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=30,
 n_iter = 100,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

param_dist = {
 'n_estimators': [128,129,130,131,132],
 'learning_rate' : [0.075,0.08,0.085,0.09,0.095],
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 20,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

## Predecir labels desconocidos

Vamos a predecir los nuevos labels.

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
training = labels.merge(df_features, how='inner', on='person')

In [20]:
param = {
    'n_estimators': 128,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

In [21]:
regr_2 = AdaBoostRegressor(**param)

In [22]:
scores = cross_val_score(regr_2, training.drop('label', axis=1), training['label'], cv=10, scoring='roc_auc')

In [23]:
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.871834 (+/- 0.023430)


In [24]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [25]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [26]:
regr_2.fit(training.drop('label', axis=1), training['label'])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=0.07, loss='linear', n_estimators=128,
         random_state=None)

In [27]:
labels_to_predict['label'] = regr_2.predict(labels_to_predict_with_features)

In [28]:
# labels_to_predict.to_csv(predicciones_csv)